<a href="https://colab.research.google.com/github/Joshua250304/soccer-predictor-2025-26-season/blob/main/prediction_2025_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"joshuakidron","key":"36243d4c445fa6a337ded33b332b4dd0"}'}

In [7]:
!mkdir -p ~/.kaggle
!mv 'kaggle (2).json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d flynn28/2025-premier-league-stats-matches-salaries

Dataset URL: https://www.kaggle.com/datasets/flynn28/2025-premier-league-stats-matches-salaries
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/57.8k [00:00<?, ?B/s]
100% 57.8k/57.8k [00:00<00:00, 142MB/s]


In [9]:
!unzip 2025-premier-league-stats-matches-salaries.zip

Archive:  2025-premier-league-stats-matches-salaries.zip
  inflating: fixtures.csv            
  inflating: player_possession_stats.csv  
  inflating: player_salaries.csv     
  inflating: player_stats.csv        
  inflating: standings.csv           
  inflating: team_possession_stats.csv  
  inflating: team_salary.csv         
  inflating: team_stats.csv          


In [47]:
import pandas as pd
import numpy as np

github_csv_url = 'https://raw.githubusercontent.com/footballcsv/cache.footballdata/refs/heads/master/2023-24/eng.1.csv'
df_eng1 = pd.read_csv(github_csv_url)
df_fixtures = pd.read_csv('fixtures.csv')

def calculate_points(row, team_col, home_goals_col, away_goals_col):
    if row[home_goals_col] > row[away_goals_col]:
      # Home Win: Home Team gets 3 points, Away Team gets 0
      if row[team_col] == row['Home']:
        return 3
      elif row[team_col] == row['Away']:
        return 0
    elif row[home_goals_col] < row[away_goals_col]:
        # Away Win: Home Team gets 0 points, Away Team gets 3
        if row[team_col] == row['Home']:
          return 0
        elif row[team_col] == row['Away']:
          return 3
    else:
        # Draw: Both gets 1 point
        return 1
    return np.nan

# --- Preprocessing eng 1.csv (2023/24 season)
df_23_24 = df_eng1.copy()

# Rename columns to be consistent for easier processing
df_23_24.rename(columns= {'Team 1': 'Home', 'Team 2': 'Away'}, inplace=True)

# Split the 'FT' column into Home and Away goals
df_23_24[['HomeGoals_23_24', 'AwayGoals_23_24']] = df_23_24['FT'].str.split('-', expand=True).astype(int)


# Calculate points for all the matches
df_home_23_24 = df_23_24[['Home', 'Away', 'HomeGoals_23_24', 'AwayGoals_23_24']].copy()
df_home_23_24['Points'] = df_home_23_24.apply(
    lambda row: calculate_points(row, 'Home', 'HomeGoals_23_24', 'AwayGoals_23_24'), axis=1
)
df_home_23_24.rename(columns={'Home': 'Team'}, inplace=True)

df_away_23_24 = df_23_24[['Home', 'Away', 'HomeGoals_23_24', 'AwayGoals_23_24']].copy()
df_away_23_24['Points'] = df_away_23_24.apply(
    lambda row: calculate_points(row, 'Home', 'HomeGoals_23_24', 'AwayGoals_23_24'), axis=1
)
df_away_23_24.rename(columns={'Away': 'Team'}, inplace=True)


# Combine and aggregate 2023/24 stats
df_23_24_combined = pd.concat([
    df_home_23_24[['Team', 'Points']],
    df_away_23_24[['Team', 'Points']]
])

team_stats_23_24 = df_23_24_combined.groupby('Team').agg(
    TotalPoints_23_24=('Points', 'sum'),
    GamesPlayed_23_24=('Points', 'count')
).reset_index()

# --- Processing fixtures.csv (2024/25 season)
df_24_25 = df_fixtures.copy()

# Ensure HomeScore and AwayScore are numeric
df_24_25['HomeScore'] = pd.to_numeric(df_24_25['HomeScore'], errors='coerce')
df_24_25['AwayScore'] = pd.to_numeric(df_24_25['AwayScore'], errors='coerce')
df_24_25.dropna(subset=['HomeScore', 'AwayScore'], inplace=True)
df_24_25['HomeScore'] = df_24_25['HomeScore'].astype(int)
df_24_25['AwayScore'] = df_24_25['AwayScore'].astype(int)

# Calculate points for all matches
df_home_24_25 = df_24_25[['Home', 'Away', 'HomeScore', 'AwayScore']].copy()
df_home_24_25['Points'] = df_home_24_25.apply(
    lambda row: calculate_points(row, 'Home', 'HomeScore', 'AwayScore'), axis=1
)
df_home_24_25.rename(columns={'Home': 'Team'}, inplace=True)


df_away_24_25 =  df_24_25[['Home', 'Away', 'HomeScore', 'AwayScore']].copy()
df_away_24_25['Points'] = df_away_24_25.apply(
    lambda row: calculate_points(row, 'Away', 'HomeScore', 'AwayScore'), axis=1
)
df_away_24_25.rename(columns={'Away': 'Team'}, inplace=True)

# Combine and aggregate 2024/25 season
df_24_25_combined = pd.concat([
    df_home_24_25[['Team', 'Points']],
    df_away_24_25[['Team', 'Points']],
])

team_stats_24_25 = df_24_25_combined.groupby('Team').agg(
    TotalPoints_24_25=('Points', 'sum'),
    GamesPlayed_24_25=('Points', 'count')
).reset_index()


# --- Merge and Predict 2025/26 ---

# Define a mapping for inconsistent team names
mapping = {
    'Man City': 'Manchester City',
    'Man United': 'Manchester Utd',
    "Nott'm Forest": "Nott'ham Forest",
}

def clean_team_name(team):
    return mapping.get(team, team)

# Clean team names and re-aggregate stats
team_stats_23_24_clean = team_stats_23_24.copy()
team_stats_23_24_clean['Team'] = team_stats_23_24_clean['Team'].apply(clean_team_name)
team_stats_23_24_clean = team_stats_23_24_clean.groupby('Team').sum().reset_index()


team_stats_24_25_clean = team_stats_24_25.copy()
team_stats_24_25_clean['Team'] = team_stats_24_25_clean['Team'].apply(clean_team_name)
team_stats_24_25_clean = team_stats_24_25_clean.groupby('Team').sum().reset_index()

# Merge the two sessions stats
df_merged_clean = pd.merge(team_stats_23_24_clean, team_stats_24_25_clean, on='Team', how='outer')

# Fill NaN with 0 for teams that were not in the league in one season (relegated/promoted)
df_prediction = df_merged_clean.fillna(0)

# Calculate combined Total Points and Total Games
df_prediction['Combined_TotalPoints'] = df_prediction['TotalPoints_23_24'] + df_prediction['TotalPoints_24_25']
df_prediction['Combined_GamesPlayed'] = df_prediction['GamesPlayed_23_24'] + df_prediction['GamesPlayed_24_25']

# Filter to include only teams that played a full season in 2024/25 (38 games)
df_prediction = df_prediction[df_prediction['GamesPlayed_24_25'] == 38]

# Calculate the overall PPG
df_prediction['Avg_PPG'] = df_prediction['Combined_TotalPoints'] / df_prediction['Combined_GamesPlayed']

# Final Prediction
df_prediction_sorted = df_prediction[['Team', 'Avg_PPG']].sort_values(by='Avg_PPG', ascending=False).reset_index(drop=True)

print(f"Predicted Champion for 2025/26 Season")
print(df_prediction_sorted.head(10))

Predicted Champion for 2025/26 Season
              Team   Avg_PPG
0        Liverpool  1.947368
1          Arsenal  1.750000
2          Chelsea  1.736842
3    Newcastle Utd  1.736842
4      Aston Villa  1.723684
5  Manchester City  1.697368
6         Brighton  1.631579
7  Nott'ham Forest  1.605263
8        Brentford  1.526316
9      Bournemouth  1.526316
